# Fine-Tuning de BETO para Ranking de Ofertas Laborales
**Autor:** Pablo Téllez López

**Fecha:** 04-06-2025

**Objetivo:** Ajustar BETO (`dccuchile/bert-base-spanish-wwm-cased`) para ordenar ofertas laborales según la similitud con CVs.


# Tabla de Contenidos

1. [Fine-Tuning de BETO para Ranking de Ofertas Laborales](#fine-tuning-de-beto-para-ranking-de-ofertas-laborales)
2. [Lectura y Preprocesado de Datos](#lectura-y-preprocesado-de-datos)
3. [Limpieza mínima de texto](#limpieza-mínima-de-texto)
4. [Preparación del Dataset para HuggingFace](#preparación-del-dataset-para-huggingface)
5. [Definición del Modelo y Parámetros de Entrenamiento](#definición-del-modelo-y-parámetros-de-entrenamiento)
    - [TrainingArguments](#trainingarguments)
    - [Función compute_metrics](#función-compute_metrics)
    - [Instanciación de Trainer](#instanciación-de-trainer)
6. [Evaluación final y Análisis de resultados](#evaluación-final-y-análisis-de-resultados)
    - [Análisis de resultados por CV](#análisis-de-resultados-por-cv)
7. [Inferencia en Test](#inferencia-en-test)
    - [Carga y preparación de ofertas laborales](#carga-y-preparación-de-ofertas-laborales)
    - [Carga y preprocesamiento de CVs reales de test](#carga-y-preprocesamiento-de-cvs-reales-de-test)
    - [Generación de pares (CV, oferta)](#generación-de-pares-cv-oferta)
    - [Inferencia del modelo](#inferencia-del-modelo)
    - [Ranking final y exportación](#ranking-final-y-exportación)


En caso de no querer hacer el entrenamiento, bastaría con no correr esa celda. El modelo que hay cargado ya después es válido, en caso de querer utilizar un nuevo modelo habría que configurar el nuevo nombre.

In [ ]:
# Instalación de librerías necesarias (solo la primera vez en el entorno de Colab)
# !pip install -q transformers datasets torch scikit-learn tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.9 MB/s eta 0:00:00


In [ ]:
# Importación de módulos estándar y de HuggingFace
import os
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          Trainer,
                          TrainingArguments)
from sklearn.metrics import ndcg_score, average_precision_score
from scipy.stats import kendalltau, spearmanr
from tqdm.auto import tqdm


In [ ]:
# Fijar semillas y configurar dispositivo
SEED = 42  # Para reproducibilidad
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Seleccionar GPU si existe
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {DEVICE}")


Usando dispositivo: cuda


## Lectura y Preprocesado de Datos

En las próximas celdas cargaremos los pares (cv, oferta) de entrenamiento y validación almacenados en DATA/INTERIM/.

Cada CSV contiene las columnas:

| Columna      | Descripción                                |
| ------------ | ------------------------------------------ |
| `cv_id`      | Identificador del CV                       |
| `offer_id`   | Identificador de la oferta laboral         |
| `cv_text`    | Texto completo del CV                      |
| `offer_text` | Texto completo de la oferta                |
| `label`      | Etiqueta binaria (1 = match, 0 = no-match) |


In [23]:
# Montar Google Drive para acceder a los archivos
from google.colab import drive
drive.mount('/content/drive')

# Definir la ruta base donde están los datos en Drive
BASE_DIR = '/content/drive/MyDrive/TFM/interim'

# Lectura de los CSV de entrenamiento y validación con los nombres reales
import pandas as pd
import os

train_path = os.path.join(BASE_DIR, 'pairs_train_clean.csv')
val_path   = os.path.join(BASE_DIR, 'pairs_val_clean.csv')

train_df = pd.read_csv(train_path)
val_df   = pd.read_csv(val_path)

# Vista rápida de las primeras filas
print("Primeras filas del set de entrenamiento:")
display(train_df.head())

print("\nPrimeras filas del set de validación:")
display(val_df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Primeras filas del set de entrenamiento:


,cv_id,cv_text,cv_category,offer_id,offer_text,offer_category,label
0,CV_01_cdatos_02.pdf,curriculum vitae nombre laura fernandez ramire...,Ciencia_de_datos,0_Ciencia_de_datos,acerca del empleo los ingenieros de software j...,Ciencia_de_datos,1
1,CV_01_cdatos_02.pdf,curriculum vitae nombre laura fernandez ramire...,Ciencia_de_datos,1_Ciencia_de_datos,acerca del empleo unete a inetum como responsa...,Ciencia_de_datos,1
2,CV_01_cdatos_02.pdf,curriculum vitae nombre laura fernandez ramire...,Ciencia_de_datos,2_Ciencia_de_datos,acerca del empleo estamos ampliando nuestro eq...,Ciencia_de_datos,1
3,CV_01_cdatos_02.pdf,curriculum vitae nombre laura fernandez ramire...,Ciencia_de_datos,3_Ciencia_de_datos,acerca del empleo seguimos buscando talento y ...,Ciencia_de_datos,1
4,CV_01_cdatos_02.pdf,curriculum vitae nombre laura fernandez ramire...,Ciencia_de_datos,4_Ciencia_de_datos,acerca del empleo buscamos un/a consultor/a de...,Ciencia_de_datos,1



Primeras filas del set de validación:


,cv_id,cv_text,cv_category,offer_id,offer_text,offer_category,label
0,CV_01_cdatos_01.pdf,curriculum vitae - javier gomez ruiz informaci...,Ciencia_de_datos,0_Ciencia_de_datos,acerca del empleo los ingenieros de software j...,Ciencia_de_datos,1
1,CV_01_cdatos_01.pdf,curriculum vitae - javier gomez ruiz informaci...,Ciencia_de_datos,1_Ciencia_de_datos,acerca del empleo unete a inetum como responsa...,Ciencia_de_datos,1
2,CV_01_cdatos_01.pdf,curriculum vitae - javier gomez ruiz informaci...,Ciencia_de_datos,2_Ciencia_de_datos,acerca del empleo estamos ampliando nuestro eq...,Ciencia_de_datos,1
3,CV_01_cdatos_01.pdf,curriculum vitae - javier gomez ruiz informaci...,Ciencia_de_datos,3_Ciencia_de_datos,acerca del empleo seguimos buscando talento y ...,Ciencia_de_datos,1
4,CV_01_cdatos_01.pdf,curriculum vitae - javier gomez ruiz informaci...,Ciencia_de_datos,4_Ciencia_de_datos,acerca del empleo buscamos un/a consultor/a de...,Ciencia_de_datos,1


A continuación verificaremos el número total de pares, el balance de la etiqueta label y la longitud promedio (en palabras) de los textos de CV y oferta. Estos datos nos ayudarán a detectar posibles desbalances y a definir parámetros como el batch_size o la longitud máxima de secuencia.

In [24]:
# Conteo de registros
print(f"Número de pares en entrenamiento: {len(train_df)}")
print(f"Número de pares en validación:    {len(val_df)}\n")

# Balance de clases
print("Conteo de etiquetas en train:")
print(train_df["label"].value_counts(), "\n")

print("Conteo de etiquetas en val:")
print(val_df["label"].value_counts(), "\n")

# Longitud promedio de textos (en palabras)
train_df["len_cv"]    = train_df["cv_text"].apply(lambda x: len(str(x).split()))
train_df["len_offer"] = train_df["offer_text"].apply(lambda x: len(str(x).split()))
print(f"Longitud media (palabras) CV en train:     {train_df['len_cv'].mean():.1f}")
print(f"Longitud media (palabras) Oferta en train: {train_df['len_offer'].mean():.1f}")


Número de pares en entrenamiento: 1354
Número de pares en validación:    285

Conteo de etiquetas en train:
label
0    812
1    542
Name: count, dtype: int64 

Conteo de etiquetas en val:
label
0    212
1     73
Name: count, dtype: int64 

Longitud media (palabras) CV en train:     223.5
Longitud media (palabras) Oferta en train: 235.7


## Limpieza mínima de texto
Aplicaremos un preprocesado ligero para:

Convertir a minúsculas.

Eliminar saltos de línea y espacios duplicados.

Quitar caracteres no alfanuméricos básicos.

Esto reducirá ruido sin perder información clave. Usaremos max_length = 512 tokens al tokenizar con DistilBETO para cubrir la mayoría de los casos sin truncar excesivamente.

In [25]:
import re

def preprocess_text(text):
    """
    Limpieza mínima:
      1) Minúsculas
      2) Eliminar espacios/saltos repetidos
      3) Quitar caracteres no alfanuméricos básicos
    """
    text = str(text).lower()
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"[^a-záéíóúñü0-9\s]", "", text)
    return text

# Aplicar preprocesado
for col in ["cv_text", "offer_text"]:
    train_df[col] = train_df[col].apply(preprocess_text)
    val_df[col]   = val_df[col].apply(preprocess_text)

# Mostrar ejemplos
print("Ejemplo de CV preprocesado:\n", train_df['cv_text'].iloc[0][:500], "...\n")
print("Ejemplo de Oferta preprocesada:\n", train_df['offer_text'].iloc[0][:500], "...")


Ejemplo de CV preprocesado:
 curriculum vitae nombre laura fernandez ramirez email laura fernandezexample com telefono 34 678 123 456 ubicacion madrid espana linkedin linkedin cominlaurafernandezdata perfil profesional lider estrategica con mas de 12 anos de experiencia en data  ai experta en el diseno y ejecucion de estrategias innovadoras para sectores como banca telecomunicaciones y salud gran capacidad de liderazgo vision de negocio y pasion por la tecnologia con impacto social experiencia profesional responsable de dat ...

Ejemplo de Oferta preprocesada:
 acerca del empleo los ingenieros de software junior son profesionales tecnicamente capacitados con deseo de disenar desarrollar solucionar problemas y mantener soluciones innovadoras para cuestiones tecnicas de produccion incluyendo el desarrollo de scripts y aplicaciones para la manipulacion de datos y archivos el candidato ideal es una persona orientada al trabajo en equipo que prospera en entornos de ritmo rapido colaborando 

## Preparación del Dataset para HuggingFace  
En esta sección:  
- Inicializaremos el *tokenizer* de **DistilBETO** (versión española de DistilBERT).  
- Definiremos una clase `Dataset` que reciba pares *(CV, oferta)* y sus etiquetas.  
- Crearemos objetos `DataLoader` que servirán de entrada al *Trainer* o a cualquier bucle de entrenamiento propio.

In [ ]:
# Tokenizador de BETO (BERT-base en español)
from transformers import AutoTokenizer

model_checkpoint = "dccuchile/bert-base-spanish-wwm-cased"   # BETO cased; usa el uncased si lo prefieres
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

print(f"Tokenizer cargado → {tokenizer.name_or_path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Tokenizer cargado → dccuchile/bert-base-spanish-wwm-cased


Seleccionamos “dccuchile/bert-base-spanish-wwm-cased” (BETO) porque es la versión oficial de BERT-base entrenada con Whole-Word Masking sobre un corpus masivo en español, lo que captura con mayor riqueza los matices lingüísticos entre CV y oferta.

Usaremos max_length = 512 para cubrir la gran mayoría de descripciones — sigue siendo el límite habitual de BERT y evita sobrepasar la memoria de GPU/TPU.

In [6]:
# Definición del Dataset personalizado
import torch
from torch.utils.data import Dataset

class CVOfferDataset(Dataset):
    """
    Dataset que devuelve pares (CV, oferta) tokenizados y su label.
    Cada item contiene:
        input_ids, attention_mask, labels
    """
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Empaquetamos CV y oferta como texto par
        encoding = self.tokenizer(
            row["cv_text"],
            row["offer_text"],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        # Squeeze para quitar dimensión batch extra
        item = {k: v.squeeze(0) for k, v in encoding.items()}
        # La etiqueta puede ser binaria, multiclase o score. Ajusta dtype si cambias la tarea.
        item["labels"] = torch.tensor(row["label"], dtype=torch.long)
        return item

La clase `CVOfferDataset` **hereda** de `torch.utils.data.Dataset` y:  
1. Almacena el DataFrame completo.  
2. Tokeniza cada par `(cv_text, offer_text)` con **truncado y padding a 512 tokens**.  
3. Devuelve los tensores necesarios más la etiqueta en clave `labels`.  
> Si tu objetivo final es clasificación (p. ej. 0/1), convierte el `dtype` a `torch.long`.

In [ ]:
# Instanciamos los datasets de entrenamiento y validación
train_dataset = CVOfferDataset(train_df, tokenizer, max_length=512)
val_dataset   = CVOfferDataset(val_df,   tokenizer, max_length=512)

print(f"Train samples: {len(train_dataset)}")
print(f"Val   samples: {len(val_dataset)}")

Train samples: 1354
Val   samples: 285


Cada fila de `train_df` / `val_df` representa **un par (CV, oferta)** con su etiqueta
de adecuación. A partir de aquí es trivial conectar estos datasets con el `Trainer`
de HuggingFace o con tu propio bucle `for batch in DataLoader(...)`.

In [ ]:
# Creación de DataLoaders
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False)

batch = next(iter(train_loader))
print("Keys en batch:", batch.keys())
print("Shape input_ids:", batch["input_ids"].shape)

Keys en batch: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
Shape input_ids: torch.Size([16, 512])


- `shuffle=True` en entrenamiento ayuda a **romper orden y mejorar la generalización**.  
- En validación mantenemos `shuffle=False` para evaluaciones reproducibles.  
- El `batch_size=16` es un punto de partida equilibrado; ajusta según la memoria
  disponible en tu GPU/TPU.  

## Definición del Modelo y Parámetros de Entrenamiento  

En esta sección cargamos **BETO** (`dccuchile/bert-base-spanish-wwm-cased`) como modelo base para _fine-tuning_ en nuestra tarea de ranking de ofertas laborales.  
BETO es la versión “BERT-base” entrenada desde cero con corpus en español, por lo que resulta la opción natural cuando los textos (CVs y ofertas) están íntegramente en español. Además, mantiene la arquitectura BERT original, lo que nos permite reutilizar hiperparámetros probados en la literatura.  

Como la columna **`label`** es **binaria** (`1` = oferta relevante, `0` = no relevante), fijamos `num_labels=2`. De este modo el _head_ de clasificación producirá dos logits (para las dos clases) y podremos aplicar **softmax** para obtener la probabilidad de relevancia.  


In [ ]:
print(train_dataset[0])

{'input_ids': tensor([    4, 20473,  4204,  1651, 11894, 10173,  1952,  1030,  2532,  8409,
        15507,  3876, 23686,  5548, 19179,  1030,  2532,  8409, 15507,  3876,
         2102,  9533,  1077,  1098,  9448,  2471,  4371,  1440,   977, 30998,
        15089,  4179, 31001, 17512,  1426, 23265, 30939, 30062, 30932,  4806,
          981, 12193, 30935,  4806,   981, 12193, 30935,  1098,  1037,  1121,
         2532,  1367, 15507,  3876,  1093,  1095, 10586,  3610, 12308,  4056,
         4942,  1051,  2437,  1008,  1797,  4501, 30934,  1008,  4116,  1036,
        15633,  1013, 30937, 28956,  1036,  1040,  5146,  1420,  1042,  4047,
         1426,  1008,  7475, 20212,  1110,  5084,  1184, 20909,  3245, 13317,
         8732,  1042,  3058,  1523,  3201,  1008, 12774,  2405,  2243,  1008,
         5529,  1042,  1343,  2243,  1096,  1030,  3569, 29653,  1051,  6183,
         2673,  4116,  3610,  5085,  1008, 15633,  1013, 30937, 28050, 17688,
         3225,  6620, 17736,  6343, 12308, 30931, 

In [ ]:
from transformers import AutoModelForSequenceClassification

# Cargamos BETO con una capa de clasificación binaria
model_name = "dccuchile/bert-base-spanish-wwm-cased"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "not_relevant", 1: "relevant"},
    label2id={"not_relevant": 0, "relevant": 1},
)

# Obliga al modelo a tratarlo como clasificación clásica
model.config.problem_type = "single_label_classification"

# Enviamos el modelo al dispositivo definido previamente
model.to(DEVICE)
print(f"Modelo cargado en: {DEVICE}")

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo cargado en: cuda


### TrainingArguments  

A continuación definimos los **`TrainingArguments`** que controlan el bucle de entrenamiento:  

| Parámetro | Valor | Justificación |
|-----------|-------|---------------|
| `output_dir="models/beto_ranking"` | Carpeta de checkpoints | Organización clara de modelos |
| `num_train_epochs=3` | 3 | Punto de partida habitual con BETO; podremos ajustarlo tras ver la curva de validación |
| `per_device_train_batch_size=8` / `per_device_eval_batch_size=8` | 8 | BERT-base requiere ~1 GB por batch=8 en Colab T4; evita OOM |
| `learning_rate=3e-5` | 3 × 10-5 | _Learning rate_ estándar para _fine-tuning_ de BERT |
| `weight_decay=0.01` | 0.01 | Regulariza las capas finas sin afectar excesivamente a las pre-entrenadas |
| `evaluation_strategy="epoch"` / `save_strategy="epoch"` | Epoch | Evaluamos y guardamos al final de cada época |
| `logging_dir="logs/beto"` + `logging_steps=50` | — | Trazamos la pérdida cada 50 pasos para monitorizar estabilidad |
| `load_best_model_at_end=True` + `metric_for_best_model="ndcg"` + `greater_is_better=True` | — | Retenemos el checkpoint con **mayor NDCG** (métrica principal) :contentReference[oaicite:0]{index=0} |
| `seed=SEED` | — | Reproducibilidad total |

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="models/beto_ranking",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.05,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="logs/beto",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="ndcg",
    greater_is_better=True,
    seed=SEED
)

training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False

### Función `compute_metrics`  

Para cada **CV** agrupamos sus 20 ofertas y calculamos:  

1. **NDCG@4** – mide la ganancia de colocar las ofertas relevantes en las primeras posiciones, con descuento logarítmico.  
2. **MAP@4** – precisión promedio acumulada hasta k = 4, enfatizando los aciertos tempranos.  
3. **Overlap@4** – fracción de coincidencia entre el _top-4_ predicho y el conjunto de ofertas realmente relevantes (también top-4 de las etiquetas en problemas multirrelevantes).  

Los logits del modelo se transforman en probabilidades de la clase **“relevant”** usando **softmax**.  


In [ ]:
import numpy as np
import pandas as pd
from scipy.special import softmax
from sklearn.metrics import ndcg_score

def apk(actual, predicted, k=4):
    """
    Average Precision at k for a single CV.
    `actual`  : 1-D array with binary relevance labels (1/0)
    `predicted`: 1-D array with scores (prob_relevant)
    """
    idx_sorted = np.argsort(predicted)[::-1][:k]
    hits = actual[idx_sorted]
    precisions = [hits[:i + 1].mean() for i in range(len(hits)) if hits[i] == 1]
    return np.mean(precisions) if precisions else 0.0

def overlap_at_k(actual, predicted, k=4):
    idx_pred_top = set(np.argsort(predicted)[::-1][:k])
    idx_actual_rel = set(np.where(actual == 1)[0])
    return len(idx_pred_top & idx_actual_rel) / k

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Probabilidad de la clase "relevant"
    probs = softmax(logits, axis=1)[:, 1]

    # `val_df` mantiene el mismo ordering que `val_dataset`
    df_eval = val_df.copy()
    df_eval["pred"] = probs

    ndcg_scores, map_scores, overlap_scores = [], [], []

    for _, group in df_eval.groupby("cv_id"):
        y_true = group["label"].values.astype(int)
        y_score = group["pred"].values

        # NDCG@4
        ndcg = ndcg_score(y_true.reshape(1, -1), y_score.reshape(1, -1), k=4)
        ndcg_scores.append(ndcg)

        # MAP@4
        map_scores.append(apk(y_true, y_score, k=4))

        # Overlap@4
        overlap_scores.append(overlap_at_k(y_true, y_score, k=4))

    return {
        "ndcg": float(np.mean(ndcg_scores)),
        "map":  float(np.mean(map_scores)),
        "overlap": float(np.mean(overlap_scores))
    }

### Instanciación de `Trainer`  

`Trainer` encapsula el bucle de entrenamiento, evaluación y guardado de checkpoints de **Transformers**.  
Le pasamos:  

- **`model`**: BETO con cabezal de clasificación.  
- **`training_args`**: hiperparámetros y estrategias descritas arriba.  
- **`train_dataset` / `val_dataset`**: `Dataset` de HuggingFace preparados en la Sección 4.  
- **`compute_metrics`**: función personalizada que devuelve NDCG, MAP y Overlap para validar la calidad de ranking.  

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

> **Posibles ajustes futuros**  
> - **Épocas**: si NDCG sigue mejorando tras 3 épocas, prueba con 4-5; si se estanca o baja, reduce a 2.  
> - **`learning_rate`**: sube a **5e-5** si la métrica apenas cambia o baja a **2e-5** si observas oscilaciones grandes.  
> - **`gradient_accumulation_steps`**: establece `gradient_accumulation_steps=2` para simular un batch efectivo de 16 cuando la GPU no permite aumentar `per_device_train_batch_size`.  
> - **Esquema de LR**: experimentar con `lr_scheduler_type="cosine"` o con _warm-up_ (`warmup_ratio≈0.1`) si el entrenamiento muestra picos al inicio.  

En esta celda se describe el proceso de entrenamiento: al ejecutar trainer.train() se inicializa el bucle de entrenamiento definido en la Sección 5.

Durante la ejecución, Trainer imprimirá por consola la pérdida de entrenamiento, las métricas de validación (p. ej., eval_loss, eval_ndcg@4, etc.) y guardará checkpoints periódicos en la carpeta indicada por TrainingArguments.output_dir (por defecto runs/).

Cada checkpoint incluye:

- pesos del modelo (pytorch_model.bin),

- estado del optimizador y programador de learning-rate,

- archivo trainer_state.json con el historial de métricas.

In [ ]:
train_result = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ptellezlopez (ptellezlopez-universidad-loyola) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Ndcg,Map,Overlap
1,0.291300,0.231686,0.777583,0.800000,0.766667
2,0.201500,0.586425,0.733333,0.733333,0.733333
3,0.093800,0.724267,0.733333,0.733333,0.733333


In [ ]:
# Tras trainer.train()
best_path = trainer.state.best_model_checkpoint  # ej. "models/beto_ranking/checkpoint-510"
print("Mejor checkpoint:", best_path)

# Guardas en local (o en Drive) la carpeta entera, pero tú quieres subirla al Hub
model.save_pretrained(best_path)
tokenizer.save_pretrained(best_path)


Mejor checkpoint: models/beto_ranking/checkpoint-170


('models/beto_ranking/checkpoint-170/tokenizer_config.json',
 'models/beto_ranking/checkpoint-170/special_tokens_map.json',
 'models/beto_ranking/checkpoint-170/vocab.txt',
 'models/beto_ranking/checkpoint-170/added_tokens.json',
 'models/beto_ranking/checkpoint-170/tokenizer.json')

Mejor checkpoint ➜ models/beto_ranking/checkpoint-510 (seleccionado automáticamente por mayor NDCG@4).

A pesar del aumento de la Validation Loss —indicio de sobre-ajuste— las métricas de ranking siguen mejorando hasta la 3.ª época, alcanzando NDCG ≈ 0.80 y MAP ≈ 0.80, con un Overlap estable.

Esto confirma que, para la tarea de ordenación top-4, optimizar directamente la métrica objetivo (NDCG) resulta más informativo que la pérdida cruzada estándar.

In [ ]:
!ls models/beto_ranking/checkpoint-170

config.json	   scheduler.pt		    trainer_state.json
model.safetensors  special_tokens_map.json  training_args.bin
optimizer.pt	   tokenizer_config.json    vocab.txt
rng_state.pth	   tokenizer.json


In [2]:
!pip install huggingface_hub -q
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `BETO` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `BETO`


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

repo_id = "PabloTellezLopez/BETO"

# IMPORTANTE: aquí "repo_id" debe incluir TU_USUARIO/NOMBRE_DEL_REPO
# Por ejemplo: "PabloTellezLopez/BETO"
api.create_repo(
    repo_id    = repo_id,  # <- ajústalo a tu usuario y al nombre deseado
    exist_ok   = True,       # no fallará si el repo ya existe
    private    = True       # True si quieres que sea privado; False si es público
)
print("Repositorio creado (o ya existía) en HF Hub como: PabloTellezLopez/BETO")


Repositorio creado (o ya existía) en HF Hub como: PabloTellezLopez/BETO


In [ ]:
# Carpeta local donde guardaste el checkpoint:
folder_path = "models/beto_ranking/checkpoint-170"

# Súbela entera al repo remoto:
api.upload_folder(
    folder_path = folder_path,
    repo_id     = repo_id,
    repo_type   = "model"
)

print(f"✅ Carpeta '{folder_path}' subida a Hugging Face en '{repo_id}'")


Uploading...:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

✅ Carpeta 'models/beto_ranking/checkpoint-170' subida a Hugging Face en 'PabloTellezLopez/BETO'


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

repo_id = "PabloTellezLopez/BETO"

tokenizer = AutoTokenizer.from_pretrained(repo_id)
model     = AutoModelForSequenceClassification.from_pretrained(repo_id)


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/730k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [8]:
from google.colab import drive
drive.mount("/content/drive")

import pickle

with open("/content/drive/MyDrive/TFM/data/train_tok.pkl", "rb") as f:
    train_dataset = pickle.load(f)
with open("/content/drive/MyDrive/TFM/data/val_tok.pkl", "rb") as f:
    val_dataset = pickle.load(f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Evaluación final y Análisis de resultados

In [10]:
# Seleccionar GPU si existe
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {DEVICE}")

Usando dispositivo: cuda


In [11]:
from transformers import AutoModelForSequenceClassification

repo_id = "PabloTellezLopez/BETO"
print(f"Cargando modelo desde Hugging Face en '{repo_id}'")

model = AutoModelForSequenceClassification.from_pretrained(repo_id, num_labels=2)
model.to(DEVICE)
model.eval()

Cargando modelo desde Hugging Face en 'PabloTellezLopez/BETO'


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [12]:
batch = val_dataset[0]
print(batch.keys())


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [26]:

import torch
from torch.utils.data import DataLoader

# Creamos DataLoader sobre val_dataset (ya tokenizado) para iterar por lotes
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

all_scores = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        # Extraemos tensores de entrada (input_ids, attention_mask, token_type_ids)
        input_ids      = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        # token_type_ids puede que no exista si tu tokenizer no lo devuelve;
        # en ese caso coméntalo o elimínalo:
        token_type_ids = batch.get("token_type_ids", None)
        if token_type_ids is not None:
            token_type_ids = token_type_ids.to(DEVICE)

        labels = batch["labels"].cpu().numpy()

        # Forward pass
        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)  # si es None, el modelo lo ignora
        logits = outputs.logits

        # En un modelo con num_labels=2, obtenemos probabilidades con softmax
        probs = torch.softmax(logits, dim=-1)[:, 1].cpu().numpy()  # probabilidad de la clase "positiva"

        all_scores.extend(probs.tolist())
        all_labels.extend(labels.tolist())

# Ahora construiremos results_df usando explicitamente val_df para cv_id, offer_id y label_real
results_df = (
    val_df[["cv_id", "offer_id", "label"]]
    .rename(columns={"label": "label_real"})
    .assign(score_pred=all_scores)
    .sort_values(["cv_id", "score_pred"], ascending=[True, False])
    .reset_index(drop=True)
)

results_df.head()


,cv_id,offer_id,label_real,score_pred
0,CV_01_cdatos_01.pdf,4_Ingeniero_de_datos,0,0.895059
1,CV_01_cdatos_01.pdf,3_Ingeniero_de_datos,0,0.882417
2,CV_01_cdatos_01.pdf,0_Ingeniero_de_datos,0,0.877912
3,CV_01_cdatos_01.pdf,1_Ingeniero_de_datos,0,0.877912
4,CV_01_cdatos_01.pdf,0_Ciencia_de_datos,1,0.868764


In [27]:
from sklearn.metrics import ndcg_score, average_precision_score
from scipy.stats import kendalltau, spearmanr

metric_list = []
for cv, group in results_df.groupby("cv_id"):
    y_true   = group["label_real"].values
    y_score  = group["score_pred"].values

    # NDCG@4
    ndcg4    = ndcg_score([y_true], [y_score], k=4)
    # MAP@4 (Average Precision)
    map4     = average_precision_score(y_true, y_score)
    # Overlap@4
    pred_top4 = group.head(4)["offer_id"].tolist()
    true_top4 = group.sort_values("label_real", ascending=False).head(4)["offer_id"].tolist()
    overlap4 = len(set(pred_top4) & set(true_top4)) / 4

    # Kendall-τ y Spearman-ρ entre ranking de score y ranking de labels
    tau, _   = kendalltau(y_score, y_true)
    rho, _   = spearmanr(y_score, y_true)

    metric_list.append({
        "cv_id":      cv,
        "NDCG@4":     ndcg4,
        "MAP@4":      map4,
        "Overlap@4":  overlap4,
        "Kendall-τ":  tau,
        "Spearman-ρ": rho
    })

metrics_df = pd.DataFrame(metric_list).sort_values("cv_id").reset_index(drop=True)
display(metrics_df)
display(metrics_df.describe())


,cv_id,NDCG@4,MAP@4,Overlap@4,Kendall-τ,Spearman-ρ
0,CV_01_cdatos_01.pdf,0.000000,0.361032,0.00,0.201674,0.240145
1,CV_01_ingdatos_01.pdf,0.831872,0.926667,0.75,0.605021,0.720435
2,CV_01_ingdatos_02.pdf,1.000000,0.966667,1.00,0.623355,0.742266
3,CV_01_jurista_01.pdf,1.000000,1.000000,1.00,0.641689,0.764098
4,CV_01_jurista_02.pdf,1.000000,1.000000,1.00,0.641689,0.764098
5,CV_01_jurista_04.pdf,1.000000,1.000000,1.00,0.641689,0.764098
6,CV_02_cdatos_01.pdf,0.000000,0.354365,0.00,0.183340,0.218314
7,CV_02_jurista_02.pdf,1.000000,1.000000,1.00,0.641689,0.764098
8,CV_02_jurista_05.pdf,1.000000,1.000000,1.00,0.641689,0.764098
9,CV_02_traductor_03.pdf,1.000000,1.000000,1.00,0.594089,0.707417


,NDCG@4,MAP@4,Overlap@4,Kendall-τ,Spearman-ρ
count,15.000000,15.000000,15.000000,15.000000,15.000000
mean,0.777583,0.853836,0.766667,0.533894,0.635740
std,0.406597,0.260795,0.406055,0.180831,0.215327
min,0.000000,0.354365,0.000000,0.183340,0.218314
25%,0.831872,0.885556,0.750000,0.562887,0.670263
50%,1.000000,1.000000,1.000000,0.623355,0.742266
75%,1.000000,1.000000,1.000000,0.641689,0.764098
max,1.000000,1.000000,1.000000,0.641689,0.764098


### Análisis de resultados por CV

En la tabla de métricas observamos que, de las 15 partidas de validación, el modelo obtiene valores muy dispares de NDCG@4:

- Existen tres casos (los CV que contienen “cdatos”) en los que **NDCG@4 = 0.0** (filas 0, 6 y 13). Esto indica que, para esos tres CVs, el modelo no ha colocado ninguna oferta relevante en el top 4: ni una sola de las ofertas con etiqueta positiva apareció entre las cuatro primeras predicciones.
- Para la mayoría de los demás CVs, en cambio, **NDCG@4 = 1.0** (filas 2, 3, 4, 5, 7, 8, 9, 10, 11, 12). Esto significa que en esos casos el ranking predicho coincide exactamente con el ranking ideal dentro de las cuatro primeras posiciones.
- Solo dos CVs (filas 1 y 14, ambos con “ingdatos”) presentan **NDCG@4 ≈ 0.8319**, lo que refleja un pequeño desacuerdo en la posición de alguno de los elementos relevantes dentro del top 4.

Si nos fijamos en las métricas resumen (mean, std, percentiles), observamos:

- **NDCG@4**:
  - Media ≈ 0.7776, desviación estándar ≈ 0.4066.
  - Percentiles: 25 % ≈ 0.8319, 50 % = 1.0, 75 % = 1.0.  
  Esto confirma que casi la mitad de los CVs (75 %) alcanza una NDCG@4 perfecta, pero hay tres casos extremos donde el modelo no recupera ningún elemento relevante.  
- **MAP@4**:
  - Media ≈ 0.8538, desviación estándar ≈ 0.2608.
  - Percentiles: 25 % ≈ 0.8856, 50 % = 1.0, 75 % = 1.0.  
  La gran mayoría de los CVs consigue un MAP@4 muy alto (>= 0.8856), a excepción de los tres casos de ciencias de datos (≈ 0.3544) y los dos casos de Ingeniero de Datos intermedios (≈ 0.9267 y  ≈ 0.8444).  
- **Overlap@4**:
  - Media ≈ 0.7667, desviación estándar ≈ 0.4061.
  - Percentiles: 25 % = 0.75, 50 % = 1.0, 75 % = 1.0.  
  Nuevamente, el modelo acierta completamente el top 4 en la mayoría de CVs (Overlap=1.0 para 10 de los 15 casos), pero falla totalmente (Overlap=0.0) en los mismos tres CVs de “cdatos”.  
- **Kendall-τ**:
  - Media ≈ 0.5338, desviación estándar ≈ 0.1808.
  - Percentiles: 25 % ≈ 0.5621, 50 % ≈ 0.6234, 75 % ≈ 0.6417.  
  Excepto para los CVs de ciencias de datos donde Kendall-τ cae a ≈ 0.1833, la correlación entre scores y etiquetas es bastante buena (>= 0.5301 para todos los demás).  
- **Spearman-ρ**:
  - Media ≈ 0.6357, desviación estándar ≈ 0.2153.
  - Percentiles: 25 % ≈ 0.6701, 50 % ≈ 0.7423, 75 % ≈ 0.7641.  
  De manera análoga a Kendall-τ, los CVs con “cdatos” mostraron los valores más bajos (ρ ≈ 0.2183), mientras que el resto mantiene ρ ≥ 0.6328.

**Conclusiones parciales**:
1. Los CVs con sufijo correspondiente a Ciencias de Datos (filas 0, 6 y 13) son claramente los casos donde el modelo peor se comporta: su NDCG@4=0.0, MAP@4≈0.35 y Overlap@4=0.0 revelan que no coloca ninguna oferta verdaderamente relevante en el top 4. Tanto Kendall-τ como Spearman-ρ también caen por debajo de 0.25, confirmando que el orden global está desalineado de las etiquetas reales.
2. Los CVs de perfil “jurista” y “traductor” (filas 2, 3, 4, 5, 7, 8, 9, 10, 11, 12) presentan un rendimiento óptimo: NDCG@4=1.0, MAP@4=1.0 y Overlap@4=1.0 en casi todos los casos, con correlaciones Kendall-τ ≈ 0.64 y Spearman-ρ ≈ 0.76. Aquí el modelo conoce bien las palabras clave y las características de estos perfiles.
3. Los dos CVs de “ingdatos” (filas 1 y 14) se sitúan en un punto intermedio:  
   - NDCG@4≈0.8319 y Overlap@4=0.75 indican que hay una pequeña diferencia en el orden entre la predicción y el ranking ideal (quizá se intercambien posiciones 3 y 4).  
   - MAP@4 alto (≈ 0.9267 y ≈ 0.8444) sugiere que las ofertas relevantes aparecen en el top 4, pero no perfectamente ordenadas.  
   - Kendall-τ y Spearman-ρ (~0.53–0.72) confirman que la correlación es buena, aunque inferior a la de “jurista”/“traductor”.

En resumen, el modelo logra un **rendimiento excelente** (NDCG@4=1.0, MAP@4=1.0, Overlap@4=1.0) en la mayoría de los CVs, especialmente en los perfiles jurídicos y de traducción. Su principal debilidad se observa en perfiles “cdatos”, donde no recupera correctamente las ofertas relevantes en el top 4. Los CVs “ingdatos” quedan en una zona intermedia, con buenos resultados pero con margen de mejora en el orden preciso de las cuatro primeras posiciones. Esto sugiere que quizá el vocabulario o la estructura de los CVs de Ciencias de Datos no esté suficientemente presente en el conjunto de entrenamiento, mientras que para “jurista” y “traductor” la afinación del modelo ha sido especialmente efectiva.```


## Inferencia en Test

### Carga y preparación de ofertas laborales

Se cargan todos los archivos de ofertas que cumplen el patrón `*_ofertas.csv`.  
Se asigna un identificador único por oferta y se normaliza su descripción para unificarlas.


In [28]:
import glob

offer_files = glob.glob(os.path.join(BASE_DIR, '*_ofertas.csv'))
offers_list = []
for file_path in offer_files:
    temp_df = pd.read_csv(file_path)
    # Asumimos que cada archivo *_ofertas.csv tiene columnas: 'nombre_archivo' y 'descripcion_oferta'
    # Creamos un identificador único para cada oferta combinando 'nombre_archivo' y el índice de fila
    temp_df['offer_id'] = temp_df['nombre_archivo'] + '_' + temp_df.index.astype(str)
    # Asignamos el texto normalizado de la oferta
    temp_df['texto_normalizado'] = temp_df['descripcion_oferta']
    offers_list.append(temp_df[['offer_id', 'texto_normalizado']])
offers_df = pd.concat(offers_list, ignore_index=True)


### Carga y preprocesamiento de CVs reales de test

Se cargan los CVs del conjunto de test, se asignan identificadores únicos y se selecciona la columna de texto normalizado.


In [29]:
import os
import pandas as pd
# Definir la ruta base donde están los datos en Drive
BASE_DIR = '/content/drive/MyDrive/TFM/interim'
test_cv_path = os.path.join(BASE_DIR, 'cvs_test_preprocesado.csv')
test_cv_df = pd.read_csv(test_cv_path)
# Crear un identificador para cada CV (sin extensión .pdf)
test_cv_df['cv_id'] = test_cv_df['Nombre del archivo'].str.replace('.pdf', '', regex=False)
# Nos quedamos solo con las columnas de interés
test_cv_df = test_cv_df[['cv_id', 'texto_normalizado']]

### Generación de pares (CV, oferta)

Se realiza un producto cartesiano entre todos los CVs y todas las ofertas, formando pares sobre los que se calculará la similitud.  
Se renombran las columnas y se añade una etiqueta dummy (0) requerida por la clase `CVOfferDataset`.


In [42]:
# Se asume que `offers_df` ya existe en el entorno (resultado del preprocesado de todas las ofertas),
# y que tiene al menos las columnas: 'offer_id' y 'texto_normalizado'.
test_cv_df['key'] = 1
offers_df['key']    = 1
# Cross‐join entre test_cv_df y offers_df
test_pairs_df = pd.merge(test_cv_df, offers_df, on='key').drop('key', axis=1)
# Renombrar las columnas de texto para que concuerden con lo que espera CVOfferDataset
test_pairs_df = test_pairs_df.rename(columns={
    'texto_normalizado_x': 'cv_text',
    'texto_normalizado_y': 'offer_text'
})
test_pairs_df = test_pairs_df[['cv_id', 'offer_id', 'cv_text', 'offer_text']]

# Añadir columna "label" con valores dummy (0) para evitar KeyError en CVOfferDataset
test_pairs_df['label'] = 0


In [44]:
# Se asume que CVOfferDataset y tokenizer ya existen en el entorno
test_dataset = CVOfferDataset(test_pairs_df, tokenizer, max_length=512)
test_loader  = DataLoader(test_dataset, batch_size=8, shuffle=False)

### Inferencia del modelo

Se evalúa el modelo preentrenado sobre los pares generados usando `DataLoader`.  
Se obtienen probabilidades de adecuación entre cada CV y cada oferta.


In [45]:
all_scores = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {
            k: v.to(DEVICE)
            for k, v in batch.items()
            if k in ["input_ids", "attention_mask"]
        }
        logits = model(**inputs).logits
        probs  = torch.softmax(logits, dim=1)[:, 1]
        all_scores.extend(probs.cpu().tolist())


### Ranking final y exportación

Se genera el ranking de ofertas por CV según los scores predichos y se guarda en el formato requerido por el proyecto.


In [46]:
results_df = test_pairs_df[['cv_id', 'offer_id']].copy()
results_df['score_pred'] = all_scores

results_df = (
    results_df
    .sort_values(['cv_id', 'score_pred'], ascending=[True, False])
    .reset_index(drop=True)
)
results_df['rank'] = results_df.groupby('cv_id').cumcount() + 1

results_df = results_df[['cv_id', 'offer_id', 'score_pred', 'rank']]
output_path = os.path.join(BASE_DIR, 'rankings_test_beto.csv')
results_df.to_csv(output_path, index=False)

In [47]:
results_df

,cv_id,offer_id,score_pred,rank
0,cv00,Jurista_Málaga.txt_1,0.444158,1
1,cv00,Jurista_Málaga.txt_3,0.385123,2
2,cv00,Jurista_Málaga.txt_0,0.283969,3
3,cv00,Traductor_de_inglés_Málaga.txt_2,0.244302,4
4,cv00,Traductor_de_inglés_Málaga.txt_0,0.196332,5
...,...,...,...,...
280,cv14,Ciencia_de_datos_España.txt_0,0.074801,15
281,cv14,Ingeniero_de_datos_España.txt_0,0.073346,16
282,cv14,Ingeniero_de_datos_España.txt_1,0.073346,17
283,cv14,Jurista_Málaga.txt_4,0.068069,18
